# Contextual chatbot using Llama2 via SageMaker JumpStart and Amazon OpenSearch Serverless with Vector Engine

## Prerequisites

You will need these prerequisites in order to build the following context aware chatbot:
- An Amazon SageMaker Execution Role with IAM permission to access Amazon OpenSearch Serverless (aoss). The [in-line](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console) policy can be found [here](./IAM/sagemaker-execution-role-aoss-policy.yml). You can determine the correct SageMaker Role by running the `setup sagemaker session` cell below.
- 2 x ml.g5.2xlarge instance types for SageMaker Endpoints. Instruction on increasing your service quota can be found [here](https://aws.amazon.com/getting-started/hands-on/request-service-quota-increase/).

### Context
Previously we saw that the model told us how to to change the tire, however we had to manually provide it with the relevant data and provide the contex ourselves. We explored the approach to leverage the model availabe under Bedrock and ask questions based on it's knowledge learned during training as well as providing manual context. While that approach works with short documents or single-ton applications, it fails to scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Pattern
We can improve upon this process by implementing an architecure called Retreival Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

In this notebook we explain how to approach the pattern of Question Answering to find and leverage the documents to provide answers to the user questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](./images/Chatbot_lang.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Usecase
#### Dataset
To explain this architecture pattern we are using the documents from IRS. These documents explain topics such as:
- Original Issue Discount (OID) Instruments
- Reporting Cash Payments of Over $10,000 to IRS
- Employer's Tax Guide

#### Persona
Let's assume a persona of a layman who doesn't have an understanding of how IRS works and if some actions have implications or not.

The model will try to answer from the documents in easy language.


## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Meta Llama2 available through Amazon SageMaker Jumpstart

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: GPT-J-6B Embeddings available through Amazon SageMaker Jumpstart

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: OpenSearch available through LangChain

  In this notebook we are using Amazon OpenSearch as a vector-store to store both the embeddings and the documents. 
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Install Required Python Libraries

    **IMPORTANT**
    1. Ensure you are running Pythin 3.10+
    1. Ensure you are using the Data Science 3.0 kernel

In [ ]:
#### Require python 3.10+
!python --version

Begin by installing the required python libraries.

In [ ]:
%pip install -U pip --quiet
%pip install --upgrade sagemaker --quiet 
%pip install langchain --quiet
%pip install opensearch-py --quiet
%pip install regex --quiet
%pip install tqdm --quiet
%pip install requests_aws4auth --quiet
%pip install pypdf

## Setup Environment

In [ ]:
# Setup SageMaker Session
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker import get_execution_role


sagemaker_session = Session()
sm_execution_role = get_execution_role()

aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

In [ ]:
# Import langchain 
from langchain.document_loaders import UnstructuredHTMLLoader,BSHTMLLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.vectorstores import OpenSearchVectorSearch
from langchain import LLMChain
from langchain import SagemakerEndpoint
from langchain.prompts import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import os
import json


In [ ]:
llm_endpoint_name = "PUT YOUR ENDPOINT NAME HERE"

In [ ]:
embeddings_endpoint_name = "PUT YOUR ENDPOINT NAME HERE"

Firt of all we have to create a vector store. In this workshop we will use Amazon OpenSerach serverless.

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [ ]:
import boto3
import time
vector_store_name = 'bedrock-workshop-rag'
index_name = "bedrock-workshop-rag-index"
encryption_policy_name = "bedrock-workshop-rag-sp"
network_policy_name = "bedrock-workshop-rag-np"
access_policy_name = 'bedrock-workshop-rag-ap'
identity = boto3.client('sts').get_caller_identity()['Arn']

aoss_client = boto3.client('opensearchserverless')

security_policy = aoss_client.create_security_policy(
    name = encryption_policy_name,
    policy = json.dumps(
        {
            'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AWSOwnedKey': True
        }),
    type = 'encryption'
)

network_policy = aoss_client.create_security_policy(
    name = network_policy_name,
    policy = json.dumps(
        [
            {'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AllowFromPublic': True}
        ]),
    type = 'network'
)

collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

while True:
    status = aoss_client.list_collections(collectionFilters={'name':vector_store_name})['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'): break
    time.sleep(10)

access_policy = aoss_client.create_access_policy(
    name = access_policy_name,
    policy = json.dumps(
        [
            {
                'Rules': [
                    {
                        'Resource': ['collection/' + vector_store_name],
                        'Permission': [
                            'aoss:CreateCollectionItems',
                            'aoss:DeleteCollectionItems',
                            'aoss:UpdateCollectionItems',
                            'aoss:DescribeCollectionItems'],
                        'ResourceType': 'collection'
                    },
                    {
                        'Resource': ['index/' + vector_store_name + '/*'],
                        'Permission': [
                            'aoss:CreateIndex',
                            'aoss:DeleteIndex',
                            'aoss:UpdateIndex',
                            'aoss:DescribeIndex',
                            'aoss:ReadDocument',
                            'aoss:WriteDocument'],
                        'ResourceType': 'index'
                    }],
                'Principal': [identity],
                'Description': 'Easy data policy'}
        ]),
    type = 'data'
)

host = collection['createCollectionDetail']['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'

## Chunk your Data and Load into Amazon OpenSearch

In this section we will chunk the data into smaller documents. Chunking is a technique for splitting large texts into smaller chunks. It is an important step as it optimizes the relevance of the search query for our RAG-model. Which in turn improves the quality of the chatbot. The chunk size is dependent on factors such as the document type and model used. We have selected a `chunk_size=2000` as this is the approximate size of a paragraph. As models improve, their context window size will increase which will allow for larger chunk sizes.

In [ ]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
    "https://www.irs.gov/pub/irs-pdf/p15.pdf",
    "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [ ]:
loader = PyPDFDirectoryLoader("./data/")
documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(documents)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) // len(
    documents
)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f"Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.")
print(f"After the split we have {len(docs)} documents more than the original {len(documents)}.")
print(
    f"Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters."
)

In [ ]:
# Helper function to process document

import regex as re

def postproc(s):
    s = s.replace(u'\xa0', u' ') # no-break space 
    s = s.replace('\n', ' ') # new-line
    s = re.sub(r'\s+', ' ', s) # multiple spaces
    return s

In [ ]:
for doc in docs:
    doc.page_content = postproc(doc.page_content)

In the next step, we simply validate that document was chunked correctly by manually reviewing the first chunk.

In [ ]:
# Review the first document for correctness
docs[10]

In [ ]:
# Limit the number of total chunks to 1000
MAX_DOCS = 1000
if len(docs) > MAX_DOCS:
    docs = docs[:MAX_DOCS]

Next we extend the LangChain `SageMakerEndpointEmbeddings` Class to create a custom embeddings function that uses the `gpt-j-6b-fp16` SageMaker Endpoint you created earlier in this notebook.

In [ ]:
import time
import json
import logging
from typing import List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

logger = logging.getLogger(__name__)

# extend the SagemakerEndpointEmbeddings class from langchain to provide a custom embedding function
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(
        self, texts: List[str], chunk_size: int = 1
    ) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        st = time.time()
        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i:i + _chunk_size])
            results.extend(response)
        time_taken = time.time() - st
        logger.info(f"got results for {len(texts)} in {time_taken}s, length of embeddings list is {len(results)}")
        print(f"got results for {len(texts)} in {time_taken}s, length of embeddings list is {len(results)}")
        return results


# class for serializing/deserializing requests/responses to/from the embeddings model
class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:

        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode('utf-8') 

    def transform_output(self, output: bytes) -> str:

        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        if len(embeddings) == 1:
            return [embeddings[0]]
        return embeddings
    

def create_sagemaker_embeddings_from_js_model(embeddings_endpoint_name: str, aws_region: str) -> SagemakerEndpointEmbeddingsJumpStart:
    # all set to create the objects for the ContentHandler and 
    # SagemakerEndpointEmbeddingsJumpStart classes
    content_handler = ContentHandler()

    # note the name of the LLM Sagemaker endpoint, this is the model that we would
    # be using for generating the embeddings
    embeddings = SagemakerEndpointEmbeddingsJumpStart( 
        endpoint_name=embeddings_endpoint_name,
        region_name=aws_region, 
        content_handler=content_handler
    )
    return embeddings

We create the embeddings object and batch the creation of the document embeddings. These embeddinga are stored in Amazon OpenSearch using LangChain `OpenSearchVectorSearch`.


In [ ]:
embeddings = create_sagemaker_embeddings_from_js_model(embeddings_endpoint_name, aws_region)

In [ ]:
from tqdm import trange
from requests_aws4auth import AWS4Auth
from opensearchpy import RequestsHttpConnection

credentials = boto3.Session().get_credentials()

service="aoss"
region=aws_region

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    service,
    session_token=credentials.token
)

docsearch = OpenSearchVectorSearch.from_texts(
    texts = [d.page_content for d in docs],
    embedding=embeddings,
    opensearch_url=host,
    http_auth=awsauth,
    timeout = 300,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name
)

In [ ]:
from langchain.load.dump import dumps

query = "Is it possible that I get sentenced to jail due to failure in filings?"

results = docsearch.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs
print(dumps(results, pretty=True))

## Question Answering Over Documents 

So far, we have chunked a large document into smaller ones, created vector embedding and stored them in an Amazon OpenSearch Serverless with Vector engine. Now, we can answer questions regarding this document data.

Since we have created an index over the data, we can do a semantic search; this way only the most relevant documents required to answer the question are passed via the prompt to the Large Language Model (LLM). This allows you to save both time and money by not passing all the documents to the LLM.

The LLM used is `Llama2` via the SageMaker Endpoint created earlier.

We use LangChian **question_answering** `stuff` document chain type in this example. Further details on Document Chains can be found by visiting the [LangChain documentation, here](https://python.langchain.com/docs/modules/chains/document/)

In [ ]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json
from langchain.chains import RetrievalQA


query = "Is it possible that I get sentenced to jail due to failure in filings?"
#query = "What is a personal income tax rate?"


template = """
Answer the following QUESTION based on the CONTEXT given. If you do not know the answer and the CONTEXT doesn't contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

template_it = """
Answer the following QUESTION based on the CONTEXT given in Italian language. If you do not know the answer and the CONTEXT doesn't contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt_template = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_messages(messages: List[Dict[str, str]]) -> List[str]:
    """
    Format messages for Llama-2 chat models.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]
    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt, model_kwargs):
        base_input = [{"role" : "user", "content" : prompt}]
        optz_input = format_messages(base_input)
        input_str = json.dumps({
            "inputs" : optz_input, 
            "parameters" : {**model_kwargs}
        })
        return input_str.encode('utf-8')
    
    def transform_output(self, output):
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_text"]
    
content_handler = ContentHandler()

llm=SagemakerEndpoint(
             endpoint_name=llm_endpoint_name, 
             region_name=aws_region, 
             model_kwargs={"max_new_tokens": 400, "top_p": 1.0, "temperature": 0.1},
             endpoint_kwargs={"CustomAttributes": "accept_eula=true"},
             content_handler=content_handler
         )

llm_qa_smep_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={
        "k": 3, 
        "space_type": "cosineSimilarity",
        "space_type": "painless_scripting"
    }),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

def pretty_print(chain_op):
    question = chain_op['query']
    response = chain_op['result']
    sources = "-" + "\n-".join([f"{src.metadata['source'].split('/')[-1]} (page: {src.metadata['page']})" for src in chain_op['source_documents']])
    sources = f"""```bash{sources}"""
    stdout = f"""{response}\n\n##### Sources:\n{sources}"""
    return stdout

llm_qa_smep_chain(query)


## Clean Up

Delete the SageMaker Inference Endpoints that we created in this notebook to avoid incurring future costs. If you created an Amazon OpenSearch Serverless Collection for this example and no longer require it then delete it via the AWS Console.

In [ ]:
# Delete LLM
llm_predictor.delete_model()
llm_predictor.delete_predictor(delete_endpoint_config=True)

# Delete Embeddings Model
embed_predictor.delete_model()
embed_predictor.delete_predictor(delete_endpoint_config=True)

# Delete OpenSearch

## Conclusion

In this notebook, we used Retrieval Augmented Generation(RAG) as an optional approach to provide domain specific context to Large Language Models(LLM). We showed how to use SageMaker Jumpstart to easily build a RAG-based contextual chatbot for a financial services organization using Llama2 and Amazon OpenSearch Serverless as the Vector datastore. This method refines text generation using Llama2 by dynamically sourcing relevant context.